In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [67]:
df = pd.read_csv('train.csv', index_col=None)

In [68]:
df

,SNR,Modulation order,Modulation
0,4,0,NaN
1,8,1,BPSK
2,8,2,QPSK
3,10,2,QPSK
4,12,3,8-QAM
5,14,4,16-QAM
6,16,5,32-QAM


In [69]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = df['Modulation']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[nan 'BPSK' 'QPSK' 'QPSK' '8-QAM' '16-QAM' '32-QAM']
[5 3 4 4 2 0 1]
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
[nan]


In [70]:
df_values = df.values
df_values

array([[4, 0, nan],
       [8, 1, 'BPSK'],
       [8, 2, 'QPSK'],
       [10, 2, 'QPSK'],
       [12, 3, '8-QAM'],
       [14, 4, '16-QAM'],
       [16, 5, '32-QAM']], dtype=object)

In [76]:
df_value_ = df_values[:, 0:2]
df_value_

array([[4, 0],
       [8, 1],
       [8, 2],
       [10, 2],
       [12, 3],
       [14, 4],
       [16, 5]], dtype=object)

In [78]:
new_df = np.hstack((df_value_, onehot_encoded))

In [79]:
new_df.shape

(7, 8)

In [91]:
df_new = pd.DataFrame(new_df, columns = ['SNR', 'Modulation order', 'NaN', 'BPSK', 'QPSK', '8-QAM', '16-QAM', '32-QAM'])
df_new

,SNR,Modulation order,NaN,BPSK,QPSK,8-QAM,16-QAM,32-QAM
0,4,0,0.0,0.0,0.0,0.0,0.0,1.0
1,8,1,0.0,0.0,0.0,1.0,0.0,0.0
2,8,2,0.0,0.0,0.0,0.0,1.0,0.0
3,10,2,0.0,0.0,0.0,0.0,1.0,0.0
4,12,3,0.0,0.0,1.0,0.0,0.0,0.0
5,14,4,1.0,0.0,0.0,0.0,0.0,0.0
6,16,5,0.0,1.0,0.0,0.0,0.0,0.0


In [92]:
X = df_new.values[:, 1:].astype(float)
X

array([[0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0., 0.],
       [2., 0., 0., 0., 0., 1., 0.],
       [2., 0., 0., 0., 0., 1., 0.],
       [3., 0., 0., 1., 0., 0., 0.],
       [4., 1., 0., 0., 0., 0., 0.],
       [5., 0., 1., 0., 0., 0., 0.]])

In [93]:
y = df_new.values[:,0].astype(float)
y

array([ 4.,  8.,  8., 10., 12., 14., 16.])

In [94]:
u = np.mean(X, axis = 0) # AXIS = 0 tells us to find mean along rows
std = np.std(X, axis =0)
print(u.shape, std.shape)

(7,) (7,)


In [95]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lin_model = LinearRegression()
lin_model.fit(X, y)

LinearRegression()

In [96]:
y_predict = lin_model.predict(X)

In [97]:
y_predict

array([ 4.,  8.,  9.,  9., 12., 14., 16.])

In [105]:
y_predict = y_predict.reshape(7,1)

In [98]:
r2 = r2_score(y, y_predict)
r2

0.9798850574712644

In [99]:
original = df.values
original

array([[4, 0, nan],
       [8, 1, 'BPSK'],
       [8, 2, 'QPSK'],
       [10, 2, 'QPSK'],
       [12, 3, '8-QAM'],
       [14, 4, '16-QAM'],
       [16, 5, '32-QAM']], dtype=object)

In [106]:
final = np.hstack((original, y_predict))

In [107]:
final

array([[4, 0, nan, 4.000000000000002],
       [8, 1, 'BPSK', 8.0],
       [8, 2, 'QPSK', 9.000000000000002],
       [10, 2, 'QPSK', 9.000000000000002],
       [12, 3, '8-QAM', 12.000000000000002],
       [14, 4, '16-QAM', 14.000000000000004],
       [16, 5, '32-QAM', 16.000000000000004]], dtype=object)

In [108]:
final_df = pd.DataFrame(final, columns=['SNR', 'Modulation order', 'Modulation(AWGN)', 'SNR_predicted'])

In [109]:
final_df

,SNR,Modulation order,Modulation(AWGN),SNR_predicted
0,4,0,NaN,4.0
1,8,1,BPSK,8.0
2,8,2,QPSK,9.0
3,10,2,QPSK,9.0
4,12,3,8-QAM,12.0
5,14,4,16-QAM,14.0
6,16,5,32-QAM,16.0


In [115]:
def hypothesis(x, theta): #input is jth row of X and theta
    y_ =0.0
    n = x.shape[0]
    for i in range(n):
        y_ += (theta[i]*x[i])
    return y_

def error(X,y, theta):
    e = 0.0
    n = X.shape[0]
    
    for i in range(n):
        y_ = hypothesis(X[i], theta)
        e +=(y[i]-y_)**2 #y of current example - y_ of current example
        
    return e/n

def gradient(X,y, theta):
    m,n =X.shape
    grad =np.zeros((n, ))
    
    #for all value of j
    for j in range(n): 
        #sum over all examples
        for i in range(m): 
            y_ =hypothesis(X[i], theta)
            grad[j] += (y_ - y[i])*X[i][j]
    #out of the loop 
    return grad/m
            

def gradient_descent(X,y, learning_rate=0.2, max_epochs =300):
    m,n = X.shape
    theta =np.zeros((n,))
    error_list = []
    
    for i in range(max_epochs):
        e = error(X,y,theta)
        error_list.append(e)
    
        #Gradient Descent
        grad = gradient(X,y, theta)
        for j in range(n):
            theta[j] = theta[j] -learning_rate*grad[j]
    
    return theta, error_list

In [116]:
theta, error_list =gradient_descent(X,y)
print(theta)

[ 3.53571429 -0.14283745 -1.678296    1.3926211   4.46353819  1.9285713
  3.99933116]


In [118]:
!python -m pip install -U notebook-as-pdf
!pyppeteer-install

You should consider upgrading via the 'C:\Python\python.exe -m pip install --upgrade pip' command.



  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61085 sha256=364c1ff63cbd3eff385589f4a8c6c8a996d4f1f1b320c2c3a13b08d3ad37d569
  Stored in directory: c:\users\vivek\appdata\local\pip\cache\wheels\d9\dc\ec\72da68331f30074b9950c1737c23cb8a67484e61498bc9713d
Successfully built PyPDF2
  Attempting uninstall: certifi
    Found existing installation: certifi 2020.12.5
    Uninstalling certifi-2020.12.5:
      Successfully uninstalled certifi-2020.12.5


[INFO] Starting Chromium download.

  0%|          | 0.00/137M [00:00<?, ?b/s]
  0%|          | 133k/137M [00:00<01:42, 1.33Mb/s]
  0%|          | 584k/137M [00:00<00:42, 3.18Mb/s]
  1%|          | 1.11M/137M [00:00<00:35, 3.80Mb/s]
  1%|1         | 2.05M/137M [00:00<00:25, 5.21Mb/s]
  3%|3         | 4.25M/137M [00:00<00:14, 9.39Mb/s]
  4%|4         | 5.81M/137M [00:00<00:11, 11.2Mb/s]
  5%|5         | 6.92M/137M [00:00<00:12, 10.4Mb/s]
  6%|6         | 8.68M/137M [00:00<00:10, 12.2Mb/s]
  8%|8         | 11.3M/137M [00:01<00:07, 16.2Mb/s]
 10%|9         | 13.1M/137M [00:01<00:08, 15.4Mb/s]
 12%|#2        | 16.6M/137M [00:01<00:05, 20.9Mb/s]
 14%|#4        | 19.2M/137M [00:01<00:06, 19.5Mb/s]
 16%|#5        | 21.3M/137M [00:01<00:06, 18.9Mb/s]
 18%|#7        | 24.0M/137M [00:01<00:05, 21.0Mb/s]
 19%|#9        | 26.6M/137M [00:01<00:04, 22.4Mb/s]
 21%|##1       | 28.9M/137M [00:01<00:05, 20.6Mb/s]
 23%|##2       | 31.0M/137M [00:02<00:06, 15.2Mb/s]
 24%|##3       | 32.8M/137M [00:02<00:0